- В этом спкрипте есть график сравнения активности в кластерах

In [1188]:
import os.path as op
import mne
import numpy as np
from mne.datasets import fetch_fsaverage
from mne.minimum_norm import make_inverse_operator, apply_inverse,apply_inverse_raw, apply_inverse_epochs
import numpy as np
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV 

In [1189]:

raw=mne.io.read_raw_fif('SG02/'+'SG02_motor_clean.fif',preload=True)
stim_initial= np.genfromtxt('SG02/SG02_motor_clean_marks.csv', delimiter=',')
raw.set_eeg_reference(projection=True)


Opening raw data file SG02/SG02_motor_clean.fif...
    Range : 0 ... 93249 =      0.000 ...   372.996 secs
Ready.
Reading 0 ... 93249  =      0.000 ...   372.996 secs...


<ipython-input-1189-23a336946676>:1: RuntimeWarning: This filename (SG02/SG02_motor_clean.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw=mne.io.read_raw_fif('SG02/'+'SG02_motor_clean.fif',preload=True)


Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


<Raw | SG02_motor_clean.fif, 30 x 93250 (373.0 s), ~21.4 MB, data loaded>

In [1190]:
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)
subject='fsaverage'

0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage


In [1191]:
def all_about_forward(raw,subjects_dir,subject='fsaverage'):
    

    subject = 'fsaverage'
    trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
    src = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
    bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')
    
    montage = mne.channels.make_standard_montage('standard_1005')
    
    dic={}

    for i in range(len(raw.ch_names)):

        kek=raw.ch_names[i].replace('.','').upper()
        kek=kek.replace('Z','z')
        kek=kek.replace('FP1','Fp1')
        kek=kek.replace('FP2','Fp2')
        kek=kek.replace('FPz','Fpz')
        dic[raw.ch_names[i]]=kek
    
    
    raw.rename_channels(mapping=dic)
    raw.set_montage(montage)
    raw.set_eeg_reference(projection=True)
    events, _ = mne.events_from_annotations(raw, event_id=dict(T1=2, T2=3))

    cov = mne.compute_raw_covariance(raw)

    fwd = mne.make_forward_solution(raw.info, trans=trans, src=src,
                                    bem=bem, eeg=True, mindist=5.0, n_jobs=1)
    #print(fwd)

    # Use fwd to compute the sensitivity map for illustration purposes
    
    cov = mne.compute_raw_covariance(raw)

    
    
    return raw,cov,fwd

In [1192]:
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)
subject='fsaverage'
        
raw, cov,fwd=all_about_forward(raw,subjects_dir)        

left_hemi,right_hemi=fwd["src"]
#left_hemi

0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Using up to 1865 segments


<ipython-input-1191-90b4dd240d1d>:25: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  raw.set_eeg_reference(projection=True)


Number of samples used : 93250
[done]
Source space          : C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif
MRI -> head transform : C:\Users\CBI\anaconda3\envs\source_space\lib\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  30 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in

In [1193]:
cov_data = np.identity(30)

cov = mne.Covariance(
        cov_data, raw.info.ch_names, raw.info["bads"], raw.info["projs"], nfree=1
)



In [1194]:
inv = make_inverse_operator(raw.info, fwd, cov, loose=0., depth=0.8,
                            verbose=True)

Computing inverse operator with 30 channels.
    30 out of 30 channels remain after picking
Selected 30 channels
Creating the depth weighting matrix...
    30 EEG channels
    limit = 20485/20484 = 2.950082
    scale = 44774.4 exp = 0.8
    Picked elements from a free-orientation depth-weighting prior into the fixed-orientation one
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 6.7e-05 (2.2e-16 eps * 30 dim * 1e+10  max singular value)
    Estimated rank (eeg): 29
    EEG: rank 29 computed from 30 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance mat

In [1195]:
raw_example=raw.copy().crop(0,20)

In [1196]:
def events_matrix_from_list(stim_initial):
    events_matrix=[]
    for i in range(1,len(stim_initial)):
        if stim_initial[i]!=stim_initial[i-1]:
            events_matrix.append([i,0,str(int(stim_initial[i]))])
        
    events_matrix=np.array(events_matrix)
    
    return events_matrix.astype(int)

In [1197]:
tmin, tmax = 0., 6.
events_matrix=events_matrix_from_list(stim_initial)
event_id=[2,3,4,5,6,7]

#real epochs of eeg data
epochs = mne.Epochs(raw, events_matrix, event_id, tmin, tmax, proj=True,
                baseline=None, preload=True)

Not setting metadata
Not setting metadata
80 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 80 events and 1501 original time points ...
1 bad epochs dropped


In [13]:
epochs_rest=epochs['2'].copy()
epochs_rest.filter(7,13)

Setting up band-pass filter from 7 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter length: 413 samples (1.652 sec)



Number of events,8
Events,2: 8
Time range,0.000 – 6.000 sec
Baseline,off


In [14]:
snr = 3.0
lambda2 = 1.0 / snr ** 2
kwargs = dict(initial_time=0.08, hemi='both', subjects_dir=subjects_dir,
              size=(600, 600))

#stc = abs(apply_inverse_raw(raw_example, inv, lambda2, 'MNE', verbose=True))

stc = apply_inverse_epochs(epochs_rest, inv, lambda2, 'MNE', verbose=True)[0]
brain = stc.plot(figure=1, initial_time=0.08, hemi='both', subjects_dir=subjects_dir, colorbar=False)
brain.add_text(0.1, 0.9, 'MNE', 'title', font_size=14)


Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 8
Processing epoch : 2 / 8
Processing epoch : 3 / 8
Processing epoch : 4 / 8
Processing epoch : 5 / 8
Processing epoch : 6 / 8
Processing epoch : 7 / 8
Processing epoch : 8 / 8
[done]
Using pyvista 3d backend.

Using control points [2.27598813e-11 2.64333768e-11 6.97310083e-11]


In [15]:
stc

<SourceEstimate | 20484 vertices, subject : fsaverage, tmin : 0.0 (ms), tmax : 6000.0 (ms), tstep : 4.0 (ms), data shape : (20484, 1501), ~234.7 MB>

### Решить обартную задачу в области и с преобразованием гильберта

In [21]:
labels=mne.read_labels_from_annot(subject, 
                               parc='aparc', 
                               hemi='both', 
                               surf_name='pial', 
                               annot_fname=None, 
                               regexp=None, 
                               subjects_dir=subjects_dir, 
                               #sort=True, 
                                  verbose=None)

Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot


In [22]:
labels_to_plot=labels[45]
labels_to_plot

<Label | fsaverage, 'postcentral-rh', rh : 9138 vertices>

### Пробуем

In [1198]:
def filter_data(eeg_data,low,high,fs,order=4):
    chunk=eeg_data
    n_channels=eeg_data.shape[0]
    b, a = butter(order, [low/fs*2, high/fs*2], btype='band')
    zi = np.zeros((max(len(b), len(a)) - 1, n_channels))
    #y, zi = lfilter(b, a, chunk.T, axis=0, zi=zi)
    y = filtfilt(b, a, chunk.T, axis=0)
    return y.T

In [1199]:
raw.filter?

In [1200]:
labels=mne.read_labels_from_annot('fsaverage',  parc='aparc',   hemi='both', surf_name='pial', 
                               annot_fname=None,  regexp=None,  subjects_dir=subjects_dir,   verbose=None)

Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot


In [1202]:
a=np.random.randint(0,10,(5,6))

In [1203]:
def all_about_labels(fwd,subject_dir='fsaverage',subject='fsaverage'):

        fs_dir = fetch_fsaverage(verbose=True)
        subject_dir = op.dirname(fs_dir)
        
        labels=extract_labels(subject,subject_dir)

        left_hemi,right_hemi=fwd["src"]

        sources_idx=np.r_[left_hemi["vertno"],right_hemi["vertno"]+len(left_hemi["vertno"])]
        print(sources_idx)
        sources_idx=left_hemi["vertno"]
        vertices_i_need=[]
        labels_i_need=[]
        poses=np.zeros((0,3))
        
        for i in range(len(labels)):
            labels[i].forward_vertices=np.where(np.isin(sources_idx,labels[i].vertices))[0]
            labels_i_need.append(labels[i])

            if labels[i].name[-2:]=='lh':
                vertices_i_need=np.concatenate([vertices_i_need,labels[i].forward_vertices])
                inds_in_this_label=[labels[i].vertices.tolist().index(k) for k in labels[i].forward_vertices]
                poses=np.concatenate([poses,labels[i].pos[inds_in_this_label,:]])

            elif labels[i].name[-2:]=='rh':
                vertices_i_need=np.concatenate([vertices_i_need,labels[i].forward_vertices+fwd['source_rr'].shape[0]/2])
                inds_in_this_label=[labels[i].vertices.tolist().index(k) for k in labels[i].forward_vertices ]
                poses=np.concatenate([poses,labels[i].pos[inds_in_this_label,:]])

        vertices_i_need_three_orient=threeplet(vertices_i_need)
        vertices_i_need_three_orient=np.array(vertices_i_need_three_orient).astype(int)

        return vertices_i_need_three_orient


In [1204]:
def threeplet(vertices_i_need):
    vertices_i_need_three=[]

    for i in range(len(vertices_i_need)):
        vertices_i_need_three.append(vertices_i_need[i]*3)
        vertices_i_need_three.append(vertices_i_need[i]*3+1)
        vertices_i_need_three.append(vertices_i_need[i]*3+2)

    return vertices_i_need_three

def return_gain_matrix(fwd):
    gain_matrix=fwd['sol']['data']
    return gain_matrix

def return_gain_matrix_ROI(fwd,vertices_i_need_three_orient):

    gain_matrix=return_gain_matrix(fwd)
    gain_matrix_small=gain_matrix[:,vertices_i_need_three_orient]
    return gain_matrix_small


def clusterization_of_gain(gain_matrix_small,cluster_num=30):
    X_inds=np.arange(0,gain_matrix_small.shape[1],3)
    Y_inds=X_inds+1
    Z_inds=X_inds+2

    gain_X=gain_matrix_small[:,X_inds]
    gain_Y=gain_matrix_small[:,Y_inds]
    gain_Z=gain_matrix_small[:,Z_inds]

    gain_all=np.concatenate([gain_X,gain_Y,gain_Z]).T

    km=KMeans(cluster_num)
    #km = AgglomerativeClustering(cluster_num)

    km.fit(gain_all)

    cluster_ids=km.fit_predict(gain_all)

    return cluster_ids


In [1212]:
def return_gain_matrix(fwd):
    gain_matrix=fwd['sol']['data']
    return gain_matrix


In [1206]:
from sklearn.cluster import KMeans

In [1207]:
def extract_labels(subject,subjects_dir):

    labels=mne.read_labels_from_annot(subject,  parc='aparc',   hemi='both', surf_name='pial', 
                               annot_fname=None,  regexp=None,  subjects_dir=subjects_dir,   verbose=None)

    return [labels[44],labels[45],labels[48],labels[49]]

def create_source_data(raw,fwd,inv,fs=0,mode='mean'):
    
    # Сейчас здесь только усреднение
    '''
    mode: describes data in which sense will be sent back
        'mean' -
        'full' -
    '''
    
    
    fs_dir = fetch_fsaverage(verbose=True)
    subjects_dir = op.dirname(fs_dir)
    subject='fsaverage'

    labels=extract_labels(subject,subjects_dir)
    
    print(labels)
    if type(raw) is np.ndarray:
        raw_matrix=raw
        fs=fs
    else:
        raw_matrix=raw.get_data()
        fs=raw.info['sfreq']

    alpha_raw=raw.copy().filter(8,12,phase='zero')
    alpha_raw=alpha_raw.apply_hilbert()

    betta_raw=raw.copy().filter(15,25,phase='zero')
    betta_raw=betta_raw.apply_hilbert()


    sources_list=[]
    for i in range(len(labels)):
        label=labels[i]
        #source_alpha= np.mean(abs(apply_inverse_raw(alpha_raw, inv, lambda2=0.01, method= 'MNE', label=label).data),0)
        #source_betta= np.mean(abs(apply_inverse_raw(betta_raw, inv, lambda2=0.01, method= 'MNE', label=label).data),0)
        
        if mode=='mean':
            source_alpha= np.mean(abs(apply_inverse_epochs(alpha_raw, inv, lambda2=0.1, method= 'sLORETA', label=label)[0].data),0)
            source_betta= np.mean(abs(apply_inverse_epochs(betta_raw, inv, lambda2=0.1, method= 'sLORETA', label=label)[0].data),0)

            sources_list.append(source_alpha)
            sources_list.append(source_betta)
            
        elif mode=='full':
            
            source_alpha= abs(apply_inverse_epochs(alpha_raw, inv, lambda2=0.1, method= 'sLORETA', label=label)[0].data)
            #source_betta= abs(apply_inverse_epochs(betta_raw, inv, lambda2=0.1, method= 'sLORETA', label=label)[0].data)

            sources_list.append(source_alpha)
            #sources_list.append(source_betta)
            
            

    clusters_data=np.vstack(sources_list)

    return clusters_data

### Если делать просто среднее в области то будет вот так

In [1208]:
labels=extract_labels('fsaverage',subjects_dir)

Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot


In [1209]:
vertices=all_about_labels(fwd,subject_dir='fsaverage',subject='fsaverage')

0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[    0     1     2 ... 20481 20482 20483]


In [1213]:
gain_matrix_ROI=return_gain_matrix_ROI(fwd,vertices)

In [1214]:
clusters=clusterization_of_gain(gain_matrix_ROI,20)

In [1215]:
clusters

array([12,  2,  8, ...,  5,  5,  5])

In [1216]:
from scipy.signal import butter, lfilter, savgol_coeffs,filtfilt

In [1226]:
plt.figure()
rest=[create_source_data(epochs['2'][i].copy(), fwd,inv,fs=250 ) for i in range(len(epochs['2']))]
hand=[create_source_data(epochs['7'][i].copy(), fwd,inv,fs=250 ) for i in range(len(epochs['7']))]
#plt.plot(aaa.T,alpha=.5)
#plt.plot(stim_initial[:10000])

0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <Label | fsaverage, 'postcentral-rh', rh : 9138 vertices>, <Label | fsaverage, 'precentral-lh', lh : 10740 vertices>, <Label | fsaverage, 'precentral-rh', rh : 10705 vertices>]
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper 

    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source cova

    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <Label | fsaverage, 'postcentral-rh', rh : 9138 vertices>, <Label | fsaverage, 'precentral-lh', lh : 10740 vertices>, <Label | fsaverage, 'precentral-rh', rh : 10705 vertices>]
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) meth

[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from t

    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <Label | fsaverage, 'postcentral-rh', rh : 9138 vertices>, <Label | fsaverage, 'precentral-lh', lh : 10740 vertices>, <Label | fsaverage, 'precentral-rh', rh : 10705 vertices>]
Setting up band-pass filter from 8 - 12 

...

Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <Label | fsaverage, 'postcentral-rh', rh 

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator f

    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_d

[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from t

    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from

In [1237]:
sources_rest.shape

(8, 8, 1501)

In [1232]:
sources_rest=np.array(aaa)

In [1238]:
sources_rest_mean_values=sources_rest.mean(0) #каждая строка это эпоха

In [1239]:
sources_rest_mean_values.shape

(8, 1501)

In [1240]:
sources_right=np.array(aaa)

In [1243]:
sources_right_mean_values=sources_right.mean(0)

In [1246]:
sources_right_mean_values.shape

(8, 1501)

In [1247]:
from sklearn.svm import SVC

In [1248]:
X_samples=np.vstack([sources_right_mean_values,sources_rest_mean_values])
Y=[2]*8+[7]*8

Иллюстрация

In [1249]:
plt.plot(X_samples.T)

In [ ]:
plt.imshow(np.vstack([np.hstack(sources_right),np.hstack(sources_rest)])[:,2700:2900])

Попытка обучить

In [1253]:
x_right=np.hstack(sources_right)[:,:10000]
x_rest=np.hstack(sources_rest)[:,:10000]



In [1254]:
x_right_test=np.hstack(sources_right)[:,10000:]
x_rest_test=np.hstack(sources_rest)[:,10000:]

In [1255]:
X_train=np.hstack([x_right,x_rest])

In [1256]:
Y=[7]*x_right.shape[1]+[2]*x_rest.shape[1]

In [1257]:
svc=SVC()
svc.fit(X_train.T,Y)

SVC()

In [1258]:
plt.plot(svc.predict(x_rest_test.T))

### Оценка качества в промышленном масштабе


In [1261]:
def make_data(raw,window,bias,states,Fs=160):

    try:
    
        time_samples=len(raw['Fz'][0][0])
    
    except :
        
        time_samples=len(np.arange(0,raw.shape[1]*1/Fs,1/Fs))
        
        
    list_of_states=[]
    train_data_list=[]

    for c in np.arange(0,(time_samples-(window-bias)),bias):

            sum_of_states=sum(states[c:c+window])

            if (sum_of_states%window)==0 and (sum_of_states/window)==states[c]:

                list_of_states.append(states[c])

                train_data_list.append(np.arange(c,(c+window)))


    return list_of_states, train_data_list


In [1262]:
def make_training_set(epochs_data_train,events, concat_flg=True):

    X_all=[]
    Y_all=[]
    for i in range(len(epochs_data_train)):
        list_of_states_i, train_data_list_i=make_data(epochs_data_train[i],320,50,[events[i,2]]*epochs_data_train[i].shape[1])
        
        X_=[]

        for state_i in range(len(list_of_states_i)-1):
            #X_.append(np.mean(epochs_data_train[i][:,train_data_list_i[state_i]],1))
            X_.append(epochs_data_train[i][:,train_data_list_i[state_i]])

        X_all.append(X_)
        Y_all.append(list_of_states_i[:-1])
    
    X_all=np.array(X_all)
    Y_all=np.array(Y_all)
    
    if concat_flg:
        X_all=np.concatenate(X_all)
        Y_all=np.concatenate(Y_all)
    
    return X_all,Y_all

In [1263]:
import pandas as pd
from mne import concatenate_raws, events_from_annotations
from mne.io import read_raw_edf
from mne.channels import make_standard_montage 

import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC

In [1264]:
from mne.datasets import eegbci

In [1266]:

#train_idx_s=[np.arange(0,len(epochs)-10)]
#test_idx_s=[np.arange(len(epochs)-10,len(epochs))]

df=pd.DataFrame(columns=['time','window','bias','tasks','csp_len','subject','validation_i','acc'])
windows=[2]
bias=[0.05]
tasks=[ 
    
    #[3, 7, 11], #Motor execution: left vs right hand
    #[4, 8, 12], #Motor imagery: left vs right hand
    [5, 9, 13], #Motor execution: hands vs feet
    #[6, 10, 14], #Motor imagery: hands vs feet

]

tain_ranges=[
    
    [0,4],
    #[1,5],
    #[1,7],
    #[0,8]
    
]


for tr in tain_ranges:

    for t in tasks:

        for w in windows:

            for b in bias:

                for i in [1]:


                    subject = i

                    tmin, tmax = -1., 7.
                    event_id = dict(hands=2, feet=3)

                    runs = t  # motor imagery: hands vs feet


                    raw_fnames = eegbci.load_data(subject, runs)
                    raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
                    raw.rename_channels(lambda x: x.strip('.'))
                    raw.rename_channels(lambda x: x.replace('Cp','CP'))
                    raw.rename_channels(lambda x: x.replace('Fc','FC'))
                    raw.rename_channels(lambda x: x.replace('Af','AF'))
                    raw.rename_channels(lambda x: x.replace('Ft','FT'))
                    raw.rename_channels(lambda x: x.replace('Tp','TP'))
                    raw.rename_channels(lambda x: x.replace('Po','PO'))


                    montage = make_standard_montage('standard_1005')
                    raw.set_montage(montage)

                    raw.rename_channels(lambda x: x.strip('.'))


                    raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

                    events, _ = events_from_annotations(raw, event_id=dict(T1=2, T2=3))

                    picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                                       exclude='bads')



                    epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                                    baseline=None, preload=True)
                    
                    
                    train_idx_s=[np.arange(10,len(epochs)),  np.concatenate([np.arange(0,10),np.arange(20,len(epochs))]),  np.concatenate([np.arange(0,20),np.arange(30,len(epochs))])  ,np.arange(0,len(epochs)-10)]
                    test_idx_s=[np.arange(0,10)           ,  np.arange(10,20)                                           ,  np.arange(20,30)                                             ,np.arange(len(epochs)-10,len(epochs))]

                    
                    train_range=tr
                    epochs_train_tmin=train_range[0]
                    epochs_train_tmax=train_range[1]

                    epochs_train = epochs.copy().crop(tmin=epochs_train_tmin, tmax=epochs_train_tmax)
                    
                    labels = epochs.events[:, -1] - 2

                    scores = []
                    #epochs_data = epochs.get_data()
                    #epochs_data_train = epochs_train.get_data()
                    
                    epochs_data=np.array([create_source_data(epochs[i].copy(), fwd,inv,fs=160,mode='full' ) for i in range(len(epochs))])
                    
                    #Здесь мы компануем все источники в кластеры
                    epochs_data_train=epochs_data[:,:,160:-160]
                    epochs_data_train=np.array( [np.mean(epochs_data_train[:,np.where(clusters==i)[0],:],1) for i in range(20)] )
                    epochs_data_train=epochs_data_train.transpose(1,0,2)
                    


                    

Extracting EDF parameters from C:\Users\CBI\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\CBI\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from C:\Users\CBI\mne_data\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming win

    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <L


FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 265 samples (1.656 sec)

Setting up band-pass filter from 15 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 141 samples (0.881 sec)

Pr

    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source cova

    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsavera

...

    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsavera

Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <Label | fsaverage, 'postcentral-rh', rh : 9138 vertices>, <Label | fsaverage, 'precentral-lh', lh : 10740 vertices>, <Label | fsaverage, 'precentral-rh', rh : 10705 vertices>]
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 265 samples (1.656 sec)

Setting up band-pa


FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 141 samples (0.881 sec)

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use

    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source cova

MemoryError: Unable to allocate 13.2 MiB for an array with shape (674, 1281) and data type complex128

In [1265]:
epochs_data_train.shape

(42, 20, 961)

In [1267]:
print('Число эпох здесь')
epochs_data_train.shape

Число эпох здесь


(42, 20, 961)

In [1268]:
print('Число эпох в матрице ивентс')
len(epochs.events)

Число эпох в матрице ивентс


42

In [1269]:
id_2=np.where(epochs_train.events[:,2]==2)[0].tolist()

In [1270]:
id_3=np.where(epochs_train.events[:,2]==3)[0].tolist()

In [1271]:
mean_2=np.mean(np.array([epochs_data_train[i] for i in id_2]),0)

In [1272]:
mean_3=np.mean(np.array([epochs_data_train[i] for i in id_3]),0)

In [1273]:
plt.figure(figsize=(15,10))

plt.imshow(mean_2-mean_3)
plt.colorbar()

In [1274]:
plt.figure(figsize=(15,10))
plt.imshow(mean_2)
plt.show()
plt.figure(figsize=(15,10))
plt.imshow(mean_3)
plt.show()

In [1275]:
plt.figure(figsize=(15,10))
[plt.plot(mean_2[i],label=str(i)+' 2') for i in range(8)]
plt.legend()
plt.show()
plt.figure(figsize=(15,10))
[plt.plot(mean_3[i],label=str(i)+' 3') for i in range(8)]
plt.legend()
plt.show()

In [989]:
plt.figure(figsize=(15,10))
plt.imshow(epochs_data_train[0])

In [ ]:
a,b=make_training_set(np.array(epochs_data_train),epochs.events)
X_means=np.mean(a,2)
Y=b

In [ ]:
X_means.shape

In [ ]:
Y.shape
Y.reshape(-1,1).shape

In [ ]:
rs=42
c_array=np.logspace(-2,3, 5)
gamma_array=np.logspace(-3,1)
kernel_array=[ 'rbf','linear','poly']

svc_params={ 'C': c_array, 'gamma': gamma_array,'kernel':kernel_array}

svc=SVC(random_state=rs)
skf = StratifiedKFold(n_splits=4)

svc_grid=GridSearchCV(svc, svc_params, scoring='roc_auc', cv=skf)
                        

In [ ]:
svc_grid.fit(X_means,Y)

In [ ]:
svc_grid.best_score_

In [ ]:
plt.plot(svc_grid.predict(X_means))
plt.plot(Y)

### Первое примнение пакета

In [1154]:
import SourceSpaceTransformer_299 as sst

In [1156]:
SST=sst.SourceSpaceTransformer_2(1,fwd,'cov','fsaverage','ss',epochs.info,clusters,inv)

lol


In [1157]:
SST.fit([2,1])

SourceSpaceTransformer_2(cov='cov',
                         fwd={'_orig_sol': array([[-108.73198975,   61.49587709,  -31.66127443, ...,  -38.37729097,
          18.77039096,   14.98657042],
       [-128.04076892,  131.92984284,   38.37732845, ...,  -32.12365756,
          22.91121737,   22.53265745],
       [ -31.49970849,  136.79250989,  111.0762157 , ...,  -23.75126535,
          25.97510267,   29.14171004],
       ...,
       [  11.91955531,  -68.68789139,   -4.78993948, ...,...
                               'file_id': None, 'gantry_angle': None,
                               'helium_info': None, 'highpass': 7.0,
                               'hpi_meas': [], 'hpi_results': [],
                               'hpi_subsystem': None, 'kit_system_id': None,
                               'line_freq': None, 'lowpass': 30.0,
                               'meas_date': datetime.datetime(2009, 8, 12, 16, 15, tzinfo=datetime.timezone.utc),
                               'meas_id': None, '

In [1158]:
epochs

Number of events,42
Events,feet: 21hands: 21
Time range,-1.000 – 7.000 sec
Baseline,off


In [1166]:
epochs.copy().crop(0,6).get_data().shape

(42, 64, 961)

In [1178]:
drdr=SST.transform(epochs[:32].copy().crop(0,5))

0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <Label | fsaverage, 'postcentral-rh', rh : 9138 vertices>, <Label | fsaverage, 'precentral-lh', lh : 10740 vertices>, <Label | fsaverage, 'precentral-rh', rh : 10705 vertices>]
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper 

- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 265 samples (1.656 sec)

Setting up band-pass filter from 15 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 141 samples (0.881 sec)

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Pick

Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenle

Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <Label | fsaverage, 'postcentral-rh', rh 


FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 265 samples (1.656 sec)

Setting up band-pass filter from 15 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 141 samples (0.881 sec)

Pr

...

    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source cova

    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <Label | fsaverage, 'postcentral-rh', rh : 9138 vertices>, <Label | fsaverage, 'precentral-lh', lh : 10740 vertices>, <Label | fsaverage, 'precentral-rh', rh : 10705 vertices>]
Set


FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 265 samples (1.656 sec)

Setting up band-pass filter from 15 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 141 samples (0.881 sec)

Pr

    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source cova

    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsavera

In [1179]:
svc_grid.fit(drdr[0],drdr[1])

GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             estimator=SVC(random_state=42),
             param_grid={'C': array([1.00000000e-02, 1.77827941e-01, 3.16227766e+00, 5.62341325e+01,
       1.00000000e+03]),
                         'gamma': array([1.00000000e-03, 1.20679264e-03, 1.45634848e-03, 1.75751062e-03,
       2.12095089e-03, 2.55954792e-03, 3.08884360e-03, 3.72759372e-03,
       4.498432...
       1.93069773e-01, 2.32995181e-01, 2.81176870e-01, 3.39322177e-01,
       4.09491506e-01, 4.94171336e-01, 5.96362332e-01, 7.19685673e-01,
       8.68511374e-01, 1.04811313e+00, 1.26485522e+00, 1.52641797e+00,
       1.84206997e+00, 2.22299648e+00, 2.68269580e+00, 3.23745754e+00,
       3.90693994e+00, 4.71486636e+00, 5.68986603e+00, 6.86648845e+00,
       8.28642773e+00, 1.00000000e+01]),
                         'kernel': ['rbf', 'linear', 'poly']},
             scoring='roc_auc')

In [1181]:
drdr_=SST.transform(epochs[32:].copy().crop(0,5))

0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <Label | fsaverage, 'postcentral-rh', rh : 9138 vertices>, <Label | fsaverage, 'precentral-lh', lh : 10740 vertices>, <Label | fsaverage, 'precentral-rh', rh : 10705 vertices>]
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper 

- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 265 samples (1.656 sec)

Setting up band-pass filter from 15 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 141 samples (0.881 sec)

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Pick

Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenle

Processing epoch : 1 / 1
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 29 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 30 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 1
[done]
0 files missing from root.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage
Reading labels from parcellation...
   read 35 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\CBI\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
[<Label | fsaverage, 'postcentral-lh', lh : 9519 vertices>, <Label | fsaverage, 'postcentral-rh', rh 


FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 265 samples (1.656 sec)

Setting up band-pass filter from 15 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 141 samples (0.881 sec)

Pr

In [1186]:
from sklearn.metrics import accuracy_score

In [1187]:
plt.plot(svc_grid.predict(drdr_[0]))
plt.plot(drdr_[1])
accuracy_score(svc_grid.predict(drdr_[0]),drdr_[1])

0.8666666666666667